Simple neural net to represent sphere on plane ambient occlusion texture
<br/>Inputs: u, v
<br/>Output: occlusion factor

In [54]:
import keras
import imageio.v2 as imageio
import numpy as np
import random
from PIL import Image

In [62]:
# get data 
img = imageio.imread('../resources/textures/sphereTex.png')
img = img[:, :, 0] # convert to grayscale
rows = len(img) # rows
cols = len(img[0]) # cols

# normalize
img = img / 255.0

# sample data
dataset = []
hashset = set()
samples = 20000
while len(dataset) < samples:
    x = random.randint(5,cols-1)
    y = random.randint(5,rows-1)
    if ((x, y) not in hashset):
        dataset.append([x/(cols-1), 1.0 - (y/(rows-1)), img[y][x]]) # u, v, ao
        hashset.add((x, y))
dataset = np.array(dataset)

# split data
np.random.shuffle(dataset)
partition = int(0.8 * len(dataset))
X_train = dataset[:partition, :-1]
y_train = dataset[:partition, -1]
X_test = dataset[partition:, :-1]
y_test = dataset[partition:, -1]

In [63]:
# build model
model = keras.Sequential([
    keras.layers.Input((2,)), # input
    keras.layers.Dense(512, activation='relu'), # hidden
    keras.layers.Dense(1), # output
])
model.compile(optimizer="rmsprop", loss=keras.losses.MeanSquaredError(), metrics="mse")

In [64]:
# train model
history = model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
500/500 [==============================] - 0s 312us/step - loss: 0.0413 - mse: 0.0413
Epoch 2/20
500/500 [==============================] - 0s 325us/step - loss: 0.0215 - mse: 0.0215
Epoch 3/20
500/500 [==============================] - 0s 278us/step - loss: 0.0202 - mse: 0.0202
Epoch 4/20
500/500 [==============================] - 0s 279us/step - loss: 0.0192 - mse: 0.0192
Epoch 5/20
500/500 [==============================] - 0s 280us/step - loss: 0.0182 - mse: 0.0182
Epoch 6/20
500/500 [==============================] - 0s 281us/step - loss: 0.0173 - mse: 0.0173
Epoch 7/20
500/500 [==============================] - 0s 278us/step - loss: 0.0165 - mse: 0.0165
Epoch 8/20
500/500 [==============================] - 0s 285us/step - loss: 0.0158 - mse: 0.0158
Epoch 9/20
500/500 [==============================] - 0s 375us/step - loss: 0.0151 - mse: 0.0151
Epoch 10/20
500/500 [==============================] - 0s 297us/step - loss: 0.0145 - mse: 0.0145
Epoch 11/20
500/500 [========

In [69]:
# generate texture using model
dim = 64
buffer = []
for r in range(dim,0,-1):
    for c in range(dim):
        x = np.array([[c/(dim-1), r/(dim-1)]])
        y = 255*model.predict(x, verbose=0)[0][0]
        if y > 255: y = 255
        buffer.append(y)

In [ ]:
# save image
img = Image.new('L', (dim,dim))
img.putdata(buffer)
img.save('nnTexture.png')